In [1]:
import time

import gym
import accelerator_environments
import joss
import matplotlib.pyplot as plt
import numpy as np
import ocelot as oc

import accelerator_environments.envs.ares.ARESlatticeStage3v1_9 as aresstuff
from accelerator_environments import utils

initializing ocelot...


In [2]:
binning = 4
screen_resolution = (int(2448 / binning), int(2040 / binning))
meter_per_pixel = (3.5488e-6 * binning, 2.5003e-6 * binning)
screen_bin_edges = (np.linspace(-screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]+1),
                                 np.linspace(-screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]+1))


cell = utils.subcell_of(aresstuff.cell, "AREASOLA1", "Drift_ARMRBSCR1")
aresstuff.areamqzm1.k1 = 5.5
aresstuff.areamqzm2.k1 = -5.5
aresstuff.areamqzm3.k1 = 0.0
# cell = [oc.Drift(l=3.0), oc.Quadrupole(l=1, k1=0), oc.Drift(l=0), oc.Drift(l=3)]
# cell = [oc.Drift(l=3.0)]

lattice = oc.MagneticLattice(cell)

In [3]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 6.654039999999998 / 6.654039999999998 : applied: 1.6352629661560059 seconds


In [4]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
navigator.unit_step = lattice.totalLen
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 6.654039999999998 / 6.654039999999998 : applied: 1.6932520866394043 seconds


In [5]:
particles = joss.generate_particles(n=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

transfer_map = joss.ocelot_lattice_2_transfer_matrix(cell)
particles = joss.track(particles, transfer_map)
image = np.histogram2d(particles[:,0], particles[:,2],
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

0.028609037399291992 seconds
